# Anthropic Claude Messages API - Multimodal

Initialize packages

In [2]:
%pip install --upgrade pip
%pip install boto3>=1.33.2 --force-reinstall --quiet
%pip install botocore>=1.33.2 --force-reinstall --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.29 requires botocore==1.34.29, but you have botocore 1.34.56 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [17]:
import boto3
import json
import base64

bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1')

In [37]:
def generate_message(bedrock_runtime, model_id, messages, max_tokens,top_p,temp,system='' if_print=0):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p,
            "system": system
        }  
    )  
    start_time = time.time()
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    end_time = time.time()
    elapsed_time = end_time - start_time
    if if_print == 1:
        print("----------------------------------------- OutPut -----------------------------------------")
        print("Elapsed time: ", elapsed_time, "seconds")
    response_body = json.loads(response.get('body').read())

    return response_body

In [7]:
with open('test.jpeg', "rb") as image_file:
    content_image = base64.b64encode(image_file.read()).decode('utf8')

In [34]:
message_mm=[{ "role": "user","content": [{"type": "image","source": { "type": "base64","media_type": \
                                                                     "image/jpeg","data": content_image}},\
                                                                     {"type": "text","text": "请从图像中提取广告口号，仅输出广告口号，广告口号不需要翻译，并以<Response>开始，</Response>结束的格式输出"}]}]

In [41]:
data = generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=message_mm,max_tokens=4096,temp=0.5,top_p=0.9,if_print=0)

text_value = data['content'][0]['text']
response_value = text_value[text_value.find('<Response>') + len('<Response>'):text_value.find('</Response>')]

print(response_value)

one hour than you earn in a day at work!


In [46]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

----------------------------------------- OutPut -----------------------------------------
Elapsed time:  7.896224021911621 seconds


{'id': 'msg_01HN5t6xX9FcfuoNb84peyRY',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': 'Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy on the atomic and subatomic scale. It is one of the two major pillars of modern physics, alongside Einstein\'s theory of relativity.\n\nSome key principles and phenomena described by quantum mechanics include:\n\n1) Wave-particle duality: Particles like electrons can exhibit properties of both particles and waves.\n\n2) Quantum state: The state of a particle is described by a wave function, which contains all the information about the particle\'s behavior.\n\n3) Quantum superposition: A particle can exist in a combination or "superposition" of multiple states simultaneously before being measured.\n\n4) Quantum entanglement: Particles can become "entangled" where their states are linked together, even across vast distances.\n\n5) Uncertainty principle: There is a fu

In [ ]:
#Messages API with system prompt
messages = [{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

system = "Respond in a way a caveman would undestand"
generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=messages,max_tokens=512,temp=0.5,top_p=0.9,system=system)